# Examples: Lattice Model Calculations
In this notebook, we do various calculations involving binomial lattice model simulation of share prices using real-world and risk-neutral probabilities.

### Setup

#### Load external packages and my codes

In [1]:
include("Include.jl");

#### What companies do we want to look at?

In [2]:
my_ticker_array = ["AMD", "WFC", "IBM", "GS", "TSLA"];

#### Load price dataset, and model parameters

In [3]:
dataset = loaddataset();
firm_mapping_df = loadfirmmappingfile();
realworld_parameters_df = loadmodelparametersfile(MyRealWorldEquityModelParameters);
riskneutral_parameters_df = loadmodelparametersfile(MySymmetricRiskNeutralEquityModelParameters);

## Task 1: Build the Real-world and Risk-neutral Risk Premium Table

In [4]:
risk_premium_df = DataFrame(id=Int64[], ticker=String[], name=String[], 
    Sₒ = Float64[], RWE=Float64[], RNE=Float64[], ΔS = Float64[], RP = Float64[])


test_index = 1200
for ticker ∈ my_ticker_array
    
    # firm -
    firm_id = realworld_parameters_df[realworld_parameters_df.ticker .== ticker, :id][1];
    firm_dataset = dataset[firm_id];
    name_value = firm_mapping_df[firm_id, :Name];
    sector_value = firm_mapping_df[firm_id, :Sector];
    Sₒ = firm_dataset[test_index, :volume_weighted_average_price];
    S₁ = firm_dataset[test_index+1, :volume_weighted_average_price];
    
    # get real-world and risk neutral parameters -
    real_world_record_df = realworld_parameters_df[realworld_parameters_df.ticker .== ticker,:];
    risk_neutral_record_df = riskneutral_parameters_df[riskneutral_parameters_df.ticker .== ticker,:];
    
    # compute real-world expectation -
    real_world_expectation = 𝔼(Sₒ = Sₒ, u = real_world_record_df[1,:up], 
        d = real_world_record_df[1,:down], probability = real_world_record_df[1,:probability]);
    
    # compute the risk-neutral expectation -
    risk_neutral_expectation = 𝔼(Sₒ = Sₒ, u = risk_neutral_record_df[1,:up], 
        d = risk_neutral_record_df[1,:down], probability = risk_neutral_record_df[1,:probability]);
        
    # compute the risk premium -
    risk_premium = log(real_world_expectation/Sₒ) - log(risk_neutral_expectation/Sₒ)
    
    # compute the change in share price -
    ΔS = (real_world_expectation - risk_neutral_expectation);
    
    # populate the df -
    results_tuple = (
        id = firm_id,
        ticker = ticker,
        name = name_value,
        # sector = sector_value,
        Sₒ = round(Sₒ,sigdigits=4), 
        RNE = round(risk_neutral_expectation, sigdigits=4),
        RWE = round(real_world_expectation, sigdigits=4),
        ΔS = round(ΔS, sigdigits=4),
        RP = round(risk_premium*100, sigdigits=4),
        # RPSP = round((ΔS/Sₒ)*100, sigdigits=4),
        # T = test_index
    );
    
    push!(risk_premium_df, results_tuple);
end

In [5]:
# risk_premium_df |> markdown_table(String) |> clipboard

## Task 2: T-day prediction capability

In [6]:
T = 21; # let's do a 21 day prediction
max_number_of_records = 1256; # we have 1256 days of data

In [7]:
date_range_set = Set{StepRange{Int64, Int64}}();
number_of_date_ranges = 1200;
counter = 0
while (counter < number_of_date_ranges)
    
    start_index = rand(1:(max_number_of_records - T - 1))
    stop_index = start_index + T
    range(start_index, stop=stop_index, step=1) |> (x-> push!(date_range_set, x))
    
    counter = length(date_range_set);
end

println("Date range has been generated")

Date range has been generated


In [8]:
success_simulation_set = Set{Tuple{Int64, StepRange{Int64, Int64}}}()
failed_simulation_set = Set{Tuple{Int64, StepRange{Int64, Int64}}}()
failed_simulation_set = Set{Tuple{Int64, StepRange{Int64, Int64}}}()
levels = [k for k ∈ 0:(T-1)]

# adapters -
parameters = riskneutral_parameters_df;
list_of_firm_ids = realworld_parameters_df[:,:id]; 

# main -
for firm_id ∈ list_of_firm_ids
    
    # parameters -
    ū = parameters[parameters.id .== firm_id, :up] |> first;
    d̄ = parameters[parameters.id .== firm_id, :down] |> first;
    p̄ = parameters[parameters.id .== firm_id, :probability] |> first;
    
    # data for this firm
    firm_data = dataset[firm_id];
    
    date_range_set_copy = deepcopy(date_range_set)
    while (isempty(date_range_set_copy) == false)
        
        date_range = pop!(date_range_set_copy);
        start_index = first(date_range)
        stop_index = last(date_range)
        
        # set the starting price
        Sₒ = firm_data[start_index, :volume_weighted_average_price];
        
        # build the model, populate with price estimates 
        model = build(MyBinomialEquityPriceTree, (
            u = ū, d = d̄, p = p̄)) |> (x-> populate(x, Sₒ, T));
        
        expectation = 𝔼(model, levels; startindex = start_index);
        variance = Var(model, levels; startindex = start_index);
        
        L = expectation[:,2] .- 2.576*sqrt.(variance[:,2])
        U = expectation[:,2] .+ 2.576*sqrt.(variance[:,2])
        
        # check the bounds: if below L, or above U then simulation failed
        success_flag = true;
        for k ∈ 1:T
            if (firm_data[start_index+k-1,:volume_weighted_average_price] < L[k]) || (firm_data[start_index+k-1,:volume_weighted_average_price] > U[k])
                success_flag  = false;
                break;
            end
        end
        
        position_tuple = (firm_id, date_range)
        if (success_flag == true)
            push!(success_simulation_set, position_tuple);
        else
            push!(failed_simulation_set, position_tuple);
        end
    end
end

In [9]:
probability_dictionary = Dict{Int64, Float64}()
for firm_id ∈ list_of_firm_ids
    probability_dictionary[firm_id] = 0.0
end

for success_case ∈ success_simulation_set
    
    firm_id = success_case |> first;
    if (haskey(probability_dictionary, firm_id) == false)
        probability_dictionary[firm_id] = (1/number_of_date_ranges);
    else
        probability_dictionary[firm_id] += (1.0/number_of_date_ranges);
    end
end

In [10]:
probability_ticker_dictionary = Dict{String, Float64}()
for (firm_id, probability) ∈ probability_dictionary
    ticker_symbol = firm_mapping_df[firm_id,:Symbol];
    probability_ticker_dictionary[ticker_symbol] = probability
end

In [11]:
# dump the prob dictionary to disk -
path_to_prob_file = joinpath(_PATH_TO_DATA,"Prob-dictionary-99-lattice-RN-T21-N$(number_of_date_ranges).jld2");
save(path_to_prob_file, Dict("dataset" => probability_ticker_dictionary))

#### Build a success probability table

In [12]:
success_probability_df = DataFrame(id=Int64[], ticker=String[], name=String[], sector=String[], 
    probability=Float64[]);

for ticker ∈ my_ticker_array
    
    firm_id = realworld_parameters_df[realworld_parameters_df.ticker .== ticker, :id][1];
    firm_dataset = dataset[firm_id];
    name_value = firm_mapping_df[firm_id, :Name];
    sector_value = firm_mapping_df[firm_id, :Sector];
    p = probability_ticker_dictionary[ticker];
    
    # populate the df -
    results_tuple = (
        id = firm_id,
        ticker = ticker,
        name = name_value,
        sector = sector_value,
        probability = round(p, sigdigits=4)
    );
    
    push!(success_probability_df, results_tuple);
end

In [14]:
success_probability_df |> markdown_table(String) |> clipboard